# Text Classification using Scikit-learn

## Overview
In this example we will use scikit-learn to train a Naive Bayes classifier using the 20 news groupd data set that is built into scikit-learn.

## Resources
Below is the resources that we used to put together this example:
- https://scikit-learn.org/
- https://scikit-learn.org/0.16/datasets/index.html
- https://seaborn.pydata.org/
- Book : Python Machine Learning By Example - Second Edition by Yuxi Liu

## Getting Started

In [ ]:
!pip install sklearn seaborn

## Step 1: Load Training Data

"20 Newsgroup" - Scikit built in data set

- https://scikit-learn.org/0.16/modules/generated/sklearn.datasets.fetch_20newsgroups.html#sklearn.datasets.fetch_20newsgroups 
- https://github.com/scikit-learn/scikit-learn/blob/a95203b/sklearn/datasets/twenty_newsgroups.py#L151

In [ ]:
from sklearn.datasets import fetch_20newsgroups

# Use 4 of the 20 categories
sub_catergory_list = ['rec.autos', 'rec.motorcycles',
                      'rec.sport.baseball','rec.sport.hockey']

# Get training data
training_data = fetch_20newsgroups(subset='train', 
                                   categories=sub_catergory_list, 
                                   shuffle=True, download_if_missing=False)

print('Training Categories:', list(training_data.target_names))
print('Total Training Data:', len(training_data.data))

## Step 2: Data Analysis
It’s good to get an understanding of your dataset. In relation to multiple classifications categories, it is important to know the distribution of the categories so categories are then not under or overrepresented.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.distplot(training_data.target)
plt.show

## Step 3 - Converting Text to Vectors
To feed predictive models with text data, we first most turn text into vectors of numerical values suitable for statistical analysis.

We will use scikit-learn 'CountVectorizer' to create feature vectors -
- https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [ ]:
def clean_data(test):
    training_data_cleaned = []
    for data in training_data.data:
        data_cleaned = ' '.join(word for word in data.split() 
                            if letter_only(word))
        training_data_cleaned.append(data_cleaned)
    
    return training_data_cleaned
    
def letter_only(word):
    for character in word:
        if not character.isalpha():
            return False
    return True

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction import stop_words

# To ensure we don't have numbers included in our feature words we will remove them.
training_data_cleaned = clean_data(training_data)

# Create count vectorizers with cleaned data and remove stop words. 
count_vectorizer = CountVectorizer(stop_words="english", max_features=5000)
x_training_count =  count_vectorizer.fit_transform(training_data_cleaned)

print('Document-Term Matrix:' + '[n_samples, n_features]: ', x_training_count.shape)

list = count_vectorizer.get_feature_names()
print(list[:5])

## Step 4: Convert Count Matrix to normalised tf-idf

__Tf__ : term-frequency, prevent giving longer text documents more weightage than shorter documents.

__tf-idf__ : term-frequency - inverse document-frequency, intended to reflect how important a word <br> is to a document in a corpus.  

- https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html#sklearn.feature_extraction.text.TfidfTransformer

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer()
x_training_tfidf = tfidf_transformer.fit_transform(x_training_count)

print('Document-Term Matrix: [n_samples, n_features_new]', x_training_tfidf.shape)

## Step 5: Build NBclassifier (Naive Bayes) - Train Classifier

We will use the MultinomialNB which implements the NB algorithm for multinomially distributed data.

- https://scikit-learn.org/stable/modules/naive_bayes.html
- https://www.analyticsvidhya.com/blog/2017/09/naive-bayes-explained/

We use the MultinomialNB which implements the NB alogorithm for multinomially distributed data.

- https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html#sklearn.naive_bayes.MultinomialNB

In [ ]:
from sklearn.naive_bayes import MultinomialNB

# Fit the classifier according to x (training vectors) and y (target values)
classifier = MultinomialNB().fit(x_training_tfidf, training_data.target) 

## Step 3 - 5 can be replaced by using a sklearn pipeline

In [ ]:
from sklearn.pipeline import Pipeline
text_classifier = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', MultinomialNB()),])
classifier = text_clf.fit(training_data.data, training_data.target)

## Step 6: Test Built Classifier
Get test data from the 20 News Group data set (same categories as training data) 

In [ ]:
import numpy as np

testing_data = fetch_20newsgroups(subset='test', categories=sub_catergory_list, shuffle=True, download_if_missing=False)

print('Total Testing Data:', len(testing_data.data))

# Transform Test data vectors 
x_testing_count = count_vectorizer.transform(testing_data.data)
x_testing_tfidf = tfidf_transformer.fit_transform(x_testing_count)

# Perform prediction on test data 
predictions = classifier.predict(x_testing_tfidf)
np.mean(predictions == testing_data.target)

In [ ]:
# Perform prediction on test data 
predictions = classifier.predict(x_testing_tfidf)

np.mean(predictions == testing_data.target)

## Step 7: Evaluate Classifier Model

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

accuracy = accuracy_score(testing_data.target, predictions)
metrics_report = classification_report(testing_data.target,  predictions)
cm = confusion_matrix(testing_data.target, predictions)

In [ ]:
print("Accuracy rate: " + str(round(accuracy, 2)) + "\n")
print("Confusion Matrix Summary \n\n", cm)
print("Report: \n\n", metrics_report)

In [ ]:
plt.rcParams["figure.figsize"] = [18,18]
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm)
plt.title('Confusion matrix graph')
fig.colorbar(cax)
ax.set_xticklabels([''] + training_data.target_names)
ax.set_yticklabels([''] + training_data.target_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

## Classify John's Invoice Document

In [ ]:
extracted_auto = "Small Auto Parts Supplier Rep Supplier Katie Russ, Payment Terms Due Date Due on receipt 02-12-2019 Qty Description 1 Wheel 2 Wax 1 Battery 2 Headlight bulb 1 Clutch INVOICE Date: 01-01-2019 INVOCE#100 John Smith, Smith's Automobiles 100 Long Roadway, Belfast BT32 QQU 028 90 222222 Customer ID ABC12345 Address 23 Seymour Drive Lisburn Northern Ireland Unit Price £40.00 £10.00 £55.00 £8.00 £140.00 Subtotal Sales Tax Total Make all checks payable to Sample Test Invoice Thank you for your business! TOTAL £40.00 £20.00 £55.00 £16.00 £140.00 £271.00 £120.00 £391.00"
extracted_moto = "Big Bike Parts Supplier Rep Supplier Seamus White Big Bike Parts Payment Terms Due Date Due on receipt 12.23.2019 Qty Description 1 Wheel 2 Wax 1 Battery 2 Headlight bulb 1 Motorcycle Fork INVOICE Date: 01/01/2019 INVOICE # 100 John Smith Smith’s Automobiles 100 Long Roadway, Belfast, Co. Antrim, BT32 QQU 028 90 222222 Customer ID ABC12345 Address 55 Prospect Drive, Belfast, Co. Antrim Unit Price £40.00 £10.00 £55.00 £8.00 £140.00 Subtotal Sales Tax Total Make all checks payable to Sample Test Invoice Thank you for your business! Line Total £40.00 £20.00 £55.00 £16.00 £140.00 £271.00 £120.00 £391.00"

In [ ]:
extracted_data_list = [extracted_auto, extracted_moto]

test_count = count_vectorizer.transform(extracted_data_list)
test_tfidf = tfidf_transformer.fit_transform(test_count)

In [ ]:
test_predicition = classifier.predict(test_tfidf)

for t in test_predicition:
    print(training_data.target_names[t])